# Day 22: Reactor Reboot

https://adventofcode.com/2021/day/22

## Part 1

Operating at these extreme ocean depths has overloaded the submarine's reactor; it needs to be rebooted.

The reactor core is made up of a large 3-dimensional grid made up entirely of cubes, one cube per integer 3-dimensional coordinate (x,y,z). Each cube can be either on or off; at the start of the reboot process, they are all off. (Could it be an old model of a reactor you've seen before?)

To reboot the reactor, you just need to set all of the cubes to either on or off by following a list of reboot steps (your puzzle input). Each step specifies a cuboid (the set of all cubes that have coordinates which fall within ranges for x, y, and z) and whether to turn all of the cubes in that cuboid on or off.

For example, given these reboot steps:
```
on x=10..12,y=10..12,z=10..12
on x=11..13,y=11..13,z=11..13
off x=9..11,y=9..11,z=9..11
on x=10..10,y=10..10,z=10..10
```
The first step (on x=10..12,y=10..12,z=10..12) turns on a 3x3x3 cuboid consisting of 27 cubes:
```
    10,10,10
    10,10,11
    10,10,12
    10,11,10
    10,11,11
    10,11,12
    10,12,10
    10,12,11
    10,12,12
    11,10,10
    11,10,11
    11,10,12
    11,11,10
    11,11,11
    11,11,12
    11,12,10
    11,12,11
    11,12,12
    12,10,10
    12,10,11
    12,10,12
    12,11,10
    12,11,11
    12,11,12
    12,12,10
    12,12,11
    12,12,12
```
The second step (on x=11..13,y=11..13,z=11..13) turns on a 3x3x3 cuboid that overlaps with the first. As a result, only 19 additional cubes turn on; the rest are already on from the previous step:
```
    11,11,13
    11,12,13
    11,13,11
    11,13,12
    11,13,13
    12,11,13
    12,12,13
    12,13,11
    12,13,12
    12,13,13
    13,11,11
    13,11,12
    13,11,13
    13,12,11
    13,12,12
    13,12,13
    13,13,11
    13,13,12
    13,13,13
```
The third step (off x=9..11,y=9..11,z=9..11) turns off a 3x3x3 cuboid that overlaps partially with some cubes that are on, ultimately turning off 8 cubes:
```
    10,10,10
    10,10,11
    10,11,10
    10,11,11
    11,10,10
    11,10,11
    11,11,10
    11,11,11
```
The final step (on x=10..10,y=10..10,z=10..10) turns on a single cube, 10,10,10. After this last step, 39 cubes are on.

The initialization procedure only uses cubes that have x, y, and z positions of at least -50 and at most 50. For now, ignore cubes outside this region.

Here is a larger example:
```
on x=-20..26,y=-36..17,z=-47..7
on x=-20..33,y=-21..23,z=-26..28
on x=-22..28,y=-29..23,z=-38..16
on x=-46..7,y=-6..46,z=-50..-1
on x=-49..1,y=-3..46,z=-24..28
on x=2..47,y=-22..22,z=-23..27
on x=-27..23,y=-28..26,z=-21..29
on x=-39..5,y=-6..47,z=-3..44
on x=-30..21,y=-8..43,z=-13..34
on x=-22..26,y=-27..20,z=-29..19
off x=-48..-32,y=26..41,z=-47..-37
on x=-12..35,y=6..50,z=-50..-2
off x=-48..-32,y=-32..-16,z=-15..-5
on x=-18..26,y=-33..15,z=-7..46
off x=-40..-22,y=-38..-28,z=23..41
on x=-16..35,y=-41..10,z=-47..6
off x=-32..-23,y=11..30,z=-14..3
on x=-49..-5,y=-3..45,z=-29..18
off x=18..30,y=-20..-8,z=-3..13
on x=-41..9,y=-7..43,z=-33..15
on x=-54112..-39298,y=-85059..-49293,z=-27449..7877
on x=967..23432,y=45373..81175,z=27513..53682
```
The last two steps are fully outside the initialization procedure area; all other steps are fully within it. After executing these steps in the initialization procedure region, 590784 cubes are on.

Execute the reboot steps. Afterward, considering only cubes in the region x=-50..50,y=-50..50,z=-50..50, **how many cubes are on?**

In [1]:
from IPython.display import Markdown
import numpy as np
import itertools

In [95]:
#infile = 'test_input1.txt'
#infile = 'test_input2.txt'
infile = 'input.txt'

def get_tup(dat):
    nums = dat.split('=')[-1]
    return tuple(int(x) for x in nums.split('..'))

boundlist = list()
xmin = None; ymin = None; zmin = None
xmax = None; ymax = None; zmax = None
limit = 50
with open(infile, 'r') as fid:
    for line in fid:
        state, bounds = line.strip().split()
        x, y, z = bounds.split(',')
        xtup = get_tup(x)
        ytup = get_tup(y)
        ztup = get_tup(z)
        # Ignore anything outside of +/-limit
        if np.any(np.array(xtup) < -limit) or np.any(np.array(xtup) > limit):
            continue
        if np.any(np.array(ytup) < -limit) or np.any(np.array(ytup) > limit):
            continue
        if np.any(np.array(ztup) < -limit) or np.any(np.array(ztup) > limit):
            continue
        boundlist.append([state, xtup, ytup, ztup])
        xmin = min(xmin, xtup[0]) if xmin is not None else xtup[0]
        ymin = min(ymin, ytup[0]) if ymin is not None else ytup[0]
        zmin = min(zmin, ztup[0]) if zmin is not None else ztup[0]
        xmax = max(xmax, xtup[1]) if xmax is not None else xtup[1]
        ymax = max(ymax, ytup[1]) if ymax is not None else ytup[1]
        zmax = max(zmax, ztup[1]) if zmax is not None else ztup[1]
#boundlist

In [96]:
#reactor_size = 102
#reactor_size = 14
#reactor = np.zeros((reactor_size, reactor_size, reactor_size), dtype=np.bool)
reactor = np.zeros((zmax-zmin+1, ymax-ymin+1, xmax-xmin+1), dtype=np.bool)
reactor.shape

(93, 95, 94)

In [97]:
for state, xb, yb, zb in boundlist:
    x0, x1 = xb
    y0, y1 = yb
    z0, z1 = zb
    # x coord -> k (cols)
    # y coord -> j (rows)
    # z coord -> i (depth)
    # Convert x,y,z to i,j,k by making zero indexed
    i0, i1 = (z0 - zmin, z1 - zmin)
    j0, j1 = (y0 - ymin, y1 - ymin)
    k0, k1 = (x0 - xmin, x1 - xmin)
    rslice = reactor[i0:i1+1,j0:j1+1,k0:k1+1]
    if 'on' == state:
        # Turn on cubes with logical or
        kern = np.ones((i1-i0+1, j1-j0+1, k1-k0+1), dtype=np.bool)
        reactor[i0:i1+1,j0:j1+1,k0:k1+1] = np.logical_or(rslice, kern)
    if 'off' == state:
        # Turn off cubes with logical and
        kern = np.zeros((i1-i0+1, j1-j0+1, k1-k0+1), dtype=np.bool)
        reactor[i0:i1+1,j0:j1+1,k0:k1+1] = np.logical_and(rslice, kern)

oncubes = reactor.sum()
Markdown("The number of 'on' cubes is **{}**".format(oncubes))

The number of 'on' cubes is **596989**

## Part Two

Now that the initialization procedure is complete, you can reboot the reactor.

Starting with all cubes off, run all of the reboot steps for all cubes in the reactor.

Consider the following reboot steps:
```
on x=-5..47,y=-31..22,z=-19..33
on x=-44..5,y=-27..21,z=-14..35
on x=-49..-1,y=-11..42,z=-10..38
on x=-20..34,y=-40..6,z=-44..1
off x=26..39,y=40..50,z=-2..11
on x=-41..5,y=-41..6,z=-36..8
off x=-43..-33,y=-45..-28,z=7..25
on x=-33..15,y=-32..19,z=-34..11
off x=35..47,y=-46..-34,z=-11..5
on x=-14..36,y=-6..44,z=-16..29
on x=-57795..-6158,y=29564..72030,z=20435..90618
on x=36731..105352,y=-21140..28532,z=16094..90401
on x=30999..107136,y=-53464..15513,z=8553..71215
on x=13528..83982,y=-99403..-27377,z=-24141..23996
on x=-72682..-12347,y=18159..111354,z=7391..80950
on x=-1060..80757,y=-65301..-20884,z=-103788..-16709
on x=-83015..-9461,y=-72160..-8347,z=-81239..-26856
on x=-52752..22273,y=-49450..9096,z=54442..119054
on x=-29982..40483,y=-108474..-28371,z=-24328..38471
on x=-4958..62750,y=40422..118853,z=-7672..65583
on x=55694..108686,y=-43367..46958,z=-26781..48729
on x=-98497..-18186,y=-63569..3412,z=1232..88485
on x=-726..56291,y=-62629..13224,z=18033..85226
on x=-110886..-34664,y=-81338..-8658,z=8914..63723
on x=-55829..24974,y=-16897..54165,z=-121762..-28058
on x=-65152..-11147,y=22489..91432,z=-58782..1780
on x=-120100..-32970,y=-46592..27473,z=-11695..61039
on x=-18631..37533,y=-124565..-50804,z=-35667..28308
on x=-57817..18248,y=49321..117703,z=5745..55881
on x=14781..98692,y=-1341..70827,z=15753..70151
on x=-34419..55919,y=-19626..40991,z=39015..114138
on x=-60785..11593,y=-56135..2999,z=-95368..-26915
on x=-32178..58085,y=17647..101866,z=-91405..-8878
on x=-53655..12091,y=50097..105568,z=-75335..-4862
on x=-111166..-40997,y=-71714..2688,z=5609..50954
on x=-16602..70118,y=-98693..-44401,z=5197..76897
on x=16383..101554,y=4615..83635,z=-44907..18747
off x=-95822..-15171,y=-19987..48940,z=10804..104439
on x=-89813..-14614,y=16069..88491,z=-3297..45228
on x=41075..99376,y=-20427..49978,z=-52012..13762
on x=-21330..50085,y=-17944..62733,z=-112280..-30197
on x=-16478..35915,y=36008..118594,z=-7885..47086
off x=-98156..-27851,y=-49952..43171,z=-99005..-8456
off x=2032..69770,y=-71013..4824,z=7471..94418
on x=43670..120875,y=-42068..12382,z=-24787..38892
off x=37514..111226,y=-45862..25743,z=-16714..54663
off x=25699..97951,y=-30668..59918,z=-15349..69697
off x=-44271..17935,y=-9516..60759,z=49131..112598
on x=-61695..-5813,y=40978..94975,z=8655..80240
off x=-101086..-9439,y=-7088..67543,z=33935..83858
off x=18020..114017,y=-48931..32606,z=21474..89843
off x=-77139..10506,y=-89994..-18797,z=-80..59318
off x=8476..79288,y=-75520..11602,z=-96624..-24783
on x=-47488..-1262,y=24338..100707,z=16292..72967
off x=-84341..13987,y=2429..92914,z=-90671..-1318
off x=-37810..49457,y=-71013..-7894,z=-105357..-13188
off x=-27365..46395,y=31009..98017,z=15428..76570
off x=-70369..-16548,y=22648..78696,z=-1892..86821
on x=-53470..21291,y=-120233..-33476,z=-44150..38147
off x=-93533..-4276,y=-16170..68771,z=-104985..-24507
```
After running the above reboot steps, 2758514936282235 cubes are on. (Just for fun, 474140 of those are also in the initialization procedure region.)

Starting again with all cubes off, execute all reboot steps. Afterward, considering all cubes, **how many cubes are on?**

In [1]:
from IPython.display import Markdown
import numpy as np
import itertools

In [4]:
#infile = 'test_input1.txt'
infile = 'test_input2.txt'
#infile = 'test_input3.txt'
#infile = 'input.txt'

def get_tup(dat):
    nums = dat.split('=')[-1]
    return tuple(int(x) for x in nums.split('..'))

boundlist = list()
#limit = 50
limit = None
with open(infile, 'r') as fid:
    for line in fid:
        state, bounds = line.strip().split()
        x, y, z = bounds.split(',')
        xtup = get_tup(x)
        ytup = get_tup(y)
        ztup = get_tup(z)
        if limit is not None:
            # Ignore anything outside of +/-limit
            if np.any(np.array(xtup) < -limit) or np.any(np.array(xtup) > limit):
                continue
            if np.any(np.array(ytup) < -limit) or np.any(np.array(ytup) > limit):
                continue
            if np.any(np.array(ztup) < -limit) or np.any(np.array(ztup) > limit):
                continue
        boundlist.append([state, xtup, ytup, ztup])
#boundlist

In [ ]:
# Represent N-dimensional data as dictionary of keys
# If key does not exist in dictionary, assume value there is False
# If key does exists, then we can get it's value directly
# We can also consider "turning off" by deleting the key from the dictionary

def get_keys(xtup, ytup, ztup):
    # From supplied x/y/z low to high tuples,
    # generate out all x,y,z tuples for the cuboid
    ii = itertools.product(range(xtup[0], xtup[1] + 1),
                           range(ytup[0], ytup[1] + 1),
                           range(ztup[0], ztup[1] + 1))
    return set(ii)

# Initialize reactor as empty set
reactor = set()
niters = len(boundlist)
i = 1
for state, xb, yb, zb in boundlist:
    print('{}/{}...'.format(i, niters), end='')
    ctups = get_keys(xb, yb, zb)
    if 'on' == state:
        # Add keys to set
        reactor |= ctups
    else:
        # Remove keys from set
        reactor -= ctups
    i+=1
    
oncubes = len(reactor)
Markdown("The number of 'on' cubes is **{}**".format(oncubes))

1/22...2/22...3/22...4/22...5/22...6/22...7/22...

In [5]:
'''
# Initialize reactor as empty dictionary
reactor = dict()

niters = len(boundlist)
i = 1
for state, xb, yb, zb in boundlist:
    print('{}/{}...'.format(i, niters), end='')
    ctups = get_keys(xb, yb, zb)
    if 'on' == state:
        # Turn on all keys
        for tup in ctups:
            reactor[tup] = 1
    if 'off' == state:
        # Delete any matching keys that are "on"
        keykill = ctups & reactor.keys()
        for tup in keykill:
            reactor.pop(tup)
    i += 1

oncubes = sum(reactor.values())
'''
#Markdown("The number of 'on' cubes is **{}**".format(oncubes))

1/60...2/60...3/60...4/60...5/60...6/60...7/60...8/60...9/60...10/60...11/60...12/60...13/60...14/60...15/60...16/60...17/60...18/60...19/60...20/60...21/60...22/60...23/60...24/60...25/60...26/60...27/60...28/60...29/60...30/60...31/60...32/60...33/60...34/60...35/60...36/60...37/60...38/60...39/60...40/60...41/60...42/60...43/60...44/60...45/60...46/60...47/60...48/60...49/60...50/60...51/60...52/60...53/60...54/60...55/60...56/60...57/60...58/60...59/60...60/60...

'\n# Initialize reactor as empty dictionary\nreactor = dict()\n\nniters = len(boundlist)\ni = 1\nfor state, xb, yb, zb in boundlist:\n    print(\'{}/{}...\'.format(i, niters), end=\'\')\n    ctups = get_keys(xb, yb, zb)\n    if \'on\' == state:\n        # Turn on all keys\n        for tup in ctups:\n            reactor[tup] = 1\n    if \'off\' == state:\n        # Delete any matching keys that are "on"\n        keykill = ctups & reactor.keys()\n        for tup in keykill:\n            reactor.pop(tup)\n    i += 1\n\noncubes = sum(reactor.values())\n'

In [1]:
# Just indexing only the on cubes is too much memory/overhead
# Probably need to represent only cuboid boundaries to be broken
# apart logically into sub-cuboids as needed. Then the volume of
# the remaining cuboid boundaries will be the answer

# Should be straightforward to solve with a constructive solid geometry
# library or openscad